# COQ4
todo

In [1]:
import gpsea
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2023-10-09
Using gpsea version 0.5.1.dev0


# COQ4
We user the [Matched Annotation from NCBI and EMBL-EBI (MANE)](https://www.ncbi.nlm.nih.gov/refseq/MANE/) transcript and the corresponding protein identifier for COQ4.

In [2]:
gene_symbol = 'COQ4'
mane_tx_id = 'NM_016035.5'
mane_protein_id = 'NP_057119.3' # ubiquinone biosynthesis protein COQ4 homolog, mitochondrial isoform 1 precursor [Homo sapiens]

In [3]:
from pyphetools.visualization import PhenopacketIngestor
pp_dir = '/Users/robin/GIT/phenopacket-store/notebooks/COQ4/phenopackets'
ingestor = PhenopacketIngestor(indir=pp_dir)
phenopackets = ingestor.get_phenopacket_list()

print(f'Loaded {len(phenopackets)} phenopackets')

[pyphetools] Ingested 42 GA4GH phenopackets.
Loaded 42 phenopackets


In [4]:
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets

cohort_creator = configure_caching_cohort_creator(hpo)

cohort, qc = load_phenopackets(
    phenopackets=phenopackets, 
    cohort_creator=cohort_creator,
)
print(f'Loaded {len(cohort)} individuals')
qc.summarize()

Individuals Processed: 100%|██████████| 42/42 [00:54<00:00,  1.29s/individuals]
Loaded 42 individuals
Validated under permissive policy
No errors or warnings were found


In [5]:
from gpsea.view import CohortViewable
from IPython.display import HTML, display

viewer = CohortViewable(hpo)
viewer.process(cohort=cohort, transcript_id=mane_tx_id)

HPO Term,ID,Seen in n individuals
Increased serum lactate,HP:0002151,20
Lower limb hyperreflexia,HP:0002395,14
Hypotonia,HP:0001252,14
Global developmental delay,HP:0001263,13
Lower limb spasticity,HP:0002061,12
Cerebellar hypoplasia,HP:0001321,11
Seizure,HP:0001250,11
Babinski sign,HP:0003487,10
Feeding difficulties,HP:0011968,10
Hearing impairment,HP:0000365,10


In [8]:
from gpsea.model.genome import GRCh38
from gpsea.preprocessing import VVMultiCoordinateService, \
    configure_default_protein_metadata_service

txc_service = VVMultiCoordinateService(genome_build=GRCh38)
tx_coordinates = txc_service.fetch(mane_tx_id) 



In [9]:
pms = configure_default_protein_metadata_service()
protein_meta = pms.annotate(mane_protein_id)
from gpsea.view import ProteinVisualizable, ProteinVisualizer

pvis = ProteinVisualizable(tx_coordinates=tx_coordinates, protein_meta=protein_meta, cohort=cohort)
drawer = ProteinVisualizer()
drawer.draw_fig(pvis=pvis)

ValueError: Could not find an entry for NP_057119.3 in Uniprot response

In [7]:
from gpsea.view import ProteinVariantViewer
viewer = ProteinVariantViewer(protein_metadata=protein_meta, tx_id=mane_tx_id)
viewer.process(cohort=cohort)

NameError: name 'protein_meta' is not defined

In [10]:
from gpsea.analysis.predicate.phenotype import prepare_predicates_for_terms_of_interest

pheno_predicates = prepare_predicates_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
    missing_implies_excluded=False,
    min_n_of_patients_with_term=2,
)
len(pheno_predicates)
from gpsea.analysis.mtc_filter import HpoMtcFilter
mtc_filter = HpoMtcFilter.default_filter(
    hpo=hpo,
     term_frequency_threshold=0.2,
)
mtc_correction = 'fdr_bh'
mtc_alpha = 0.05
from gpsea.analysis.pcats.stats import FisherExactTest

count_statistic = FisherExactTest()
from gpsea.analysis.pcats import HpoTermAnalysis

analysis = HpoTermAnalysis(
    count_statistic=count_statistic,
    mtc_filter=mtc_filter,
    mtc_correction=mtc_correction,
    mtc_alpha=mtc_alpha,
)

In [20]:
from gpsea.model import VariantEffect
from gpsea.analysis.predicate.genotype import VariantPredicates, biallelic_predicate

is_missense = VariantPredicates.variant_effect(VariantEffect.MISSENSE_VARIANT, mane_tx_id)
moi_predicate = biallelic_predicate(
    a_predicate=is_missense,
    b_predicate=~is_missense,
    names=("missense", "other"),
    partitions=((1,0),(2,))
)
moi_predicate.display_question()

'Allele group: missense/other OR missense/missense, other/other'

In [21]:
result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=moi_predicate,
    pheno_predicates=pheno_predicates,
)
from gpsea.view import MtcStatsViewer

viewer = MtcStatsViewer()
viewer.process(result)

Code,Reason,Count
HMF01,Skipping term with maximum frequency that was less than threshold 0.2,4
HMF03,Skipping term because of a child term with the same individual counts,2
HMF05,Skipping term because one genotype had zero observations,24
HMF08,Skipping general term,51
HMF09,Skipping term with maximum annotation frequency that was less than threshold 0.25,92


In [22]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=result)

Allele group                                       missense/other OR missense/missense  \
                                                                                 Count   
Lower limb hyperreflexia [HP:0002395]                                            14/14   
Cognitive impairment [HP:0100543]                                                 1/12   
Dysarthria [HP:0001260]                                                           2/13   
Cerebellar atrophy [HP:0001272]                                                   4/20   
Limb ataxia [HP:0002070]                                                          3/12   
Gait ataxia [HP:0002066]                                                          3/12   
Babinski sign [HP:0003487]                                                       10/14   
Lower limb spasticity [HP:0002061]                                               12/20   
Morphological central nervous system abnormalit...                               21/21   
Abnormal cerebellum morphology [HP:0001317]                                      13/13   
Abnormal brain morphology [HP:0012443]                                           21/21   

Allele group                                               other/other  \
                                                   Percent       Count   
Lower limb hyperreflexia [HP:0002395]                 100%         0/2   
Cognitive impairment [HP:0100543]                       8%         2/2   
Dysarthria [HP:0001260]                                15%         2/2   
Cerebellar atrophy [HP:0001272]                        20%         2/2   
Limb ataxia [HP:0002070]                               25%         2/2   
Gait ataxia [HP:0002066]                               25%         2/2   
Babinski sign [HP:0003487]                             71%         0/2   
Lower limb spasticity [HP:0002061]                     60%         0/2   
Morphological central nervous system abnormalit...    100%         2/2   
Abnormal cerebellum morphology [HP:0001317]           100%         2/2   
Abnormal brain morphology [HP:0012443]                100%         2/2   

Allele group                                                                   \
                                                   Percent Corrected p values   
Lower limb hyperreflexia [HP:0002395]                   0%           0.091667   
Cognitive impairment [HP:0100543]                     100%           0.178571   
Dysarthria [HP:0001260]                               100%           0.178571   
Cerebellar atrophy [HP:0001272]                       100%           0.178571   
Limb ataxia [HP:0002070]                              100%           0.196429   
Gait ataxia [HP:0002066]                              100%           0.196429   
Babinski sign [HP:0003487]                              0%           0.196429   
Lower limb spasticity [HP:0002061]                      0%           0.267857   
Morphological central nervous system abnormalit...    100%           1.000000   
Abnormal cerebellum morphology [HP:0001317]           100%           1.000000   
Abnormal brain morphology [HP:0012443]                100%           1.000000   

Allele group                                                  
                                                    p values  
Lower limb hyperreflexia [HP:0002395]               0.008333  
Cognitive impairment [HP:0100543]                   0.032967  
Dysarthria [HP:0001260]                             0.057143  
Cerebellar atrophy [HP:0001272]                     0.064935  
Limb ataxia [HP:0002070]                            0.109890  
Gait ataxia [HP:0002066]                            0.109890  
Babinski sign [HP:0003487]                          0.125000  
Lower limb spasticity [HP:0002061]                  0.194805  
Morphological central nervous system abnormalit...  1.000000  
Abnormal cerebellum morphology [HP:0001317]         1.000000  
Abnormal brain morphology [HP:0012443]              1.00

In [24]:
is_g124s = VariantPredicates.variant_key("9_128325849_128325849_G_A") # c.370G>A	p.Gly124Ser
g124_predicate = biallelic_predicate(
    a_predicate=is_g124s,
    b_predicate=~is_g124s,
    names=("G124S", "other"),
    partitions=((1,0),(2,))
)
g124_predicate.display_question()

'Allele group: G124S/other OR G124S/G124S, other/other'

In [25]:
result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=g124_predicate,
    pheno_predicates=pheno_predicates,
)
summarize_hpo_analysis(hpo=hpo, result=result)

Allele group                                       G124S/other OR G124S/G124S  \
                                                                        Count   
Hypotonia [HP:0001252]                                                   6/10   
Limb ataxia [HP:0002070]                                                  0/4   
Gait ataxia [HP:0002066]                                                  0/4   
Feeding difficulties [HP:0011968]                                         6/7   
Cognitive impairment [HP:0100543]                                         0/4   
Dysarthria [HP:0001260]                                                   0/4   
Hypertrophic cardiomyopathy [HP:0001639]                                  3/6   
Hearing impairment [HP:0000365]                                           5/6   
Increased serum lactate [HP:0002151]                                    11/12   
Lower limb spasticity [HP:0002061]                                        4/6   
Spasticity [HP:0001257]                                                   7/9   
Aplasia/Hypoplasia of the cerebellum [HP:0007360]                         5/5   
Aplasia/Hypoplasia involving the central nervou...                        7/7   
Morphological central nervous system abnormalit...                        8/8   
Abnormal cerebellum morphology [HP:0001317]                               5/5   
Abnormal brain morphology [HP:0012443]                                    8/8   
Cerebellar hypoplasia [HP:0001321]                                        5/8   
Appendicular spasticity [HP:0034353]                                      4/6   
Upper motor neuron dysfunction [HP:0002493]                               7/7   
Hypertonia [HP:0001276]                                                   7/7   
Abnormal muscle tone [HP:0003808]                                       12/12   
Lower limb hyperreflexia [HP:0002395]                                     4/4   
Hyperreflexia [HP:0001347]                                                4/4   
Abnormal reflex [HP:0031826]                                              4/4   
Babinski sign [HP:0003487]                                                3/4   
Acidosis [HP:0001941]                                                   11/11   
Cerebellar atrophy [HP:0001272]                                          3/11   
Abnormal cerebral morphology [HP:0002060]                                 8/8   
Abnormal forebrain morphology [HP:0100547]                                8/8   
Thin corpus callosum [HP:0033725]                                         5/8   
Global developmental delay [HP:0001263]                                 11/11   
Neurodevelopmental delay [HP:0012758]                                   11/11   
Seizure [HP:0001250]                                                    10/12   
Cerebral visual impairment [HP:0100704]                                  4/10   

Allele group                                               other/other  \
                                                   Percent       Count   
Hypotonia [HP:0001252]                                 60%         8/8   
Limb ataxia [HP:0002070]                                0%        5/10   
Gait ataxia [HP:0002066]                                0%        5/10   
Feeding difficulties [HP:0011968]                      86%         4/8   
Cognitive impairment [HP:0100543]                       0%        3/10   
Dysarthria [HP:0001260]                                 0%        4/11   
Hypertrophic cardiomyopathy [HP:0001639]               50%         1/6   
Hearing impairment [HP:0000365]                        83%         5/8   
Increased serum lactate [HP:0002151]                   92%        9/12   
Lower limb spasticity [HP:0002061]                     67%        8/16   
Spasticity [HP:0001257]                                78%        9/14   
Aplasia/Hypoplasia of the cerebellum [HP:0007360]     100%         7/7   
Aplasia/Hypoplasia involving the central nervou